负责把iqfeed格式的1min csv合成为vnpy格式的1min csv

In [1]:
from vnpy.trader.constant import (Exchange, Interval)
import pandas as pd
import os
import glob
import datetime

In [2]:
symbol_frame = pd.read_csv(r"E:\project\trade_engine\iqfeed_data_processor\project_data\my_symbols.csv")

In [3]:
symbol_frame.head()

,description,iqfeed_roots,ib_root,symbol_exchange,commission_ratio,commission,tick_value,tick_price,size,rough_price,contract_value,init_margin,maintain_margin
0,S&P 500 E-Mini,@ES,ES,GLOBEX,0.000015,2.10,12.50,0.25,50,2841.0,142050,13200.00,12000.00
1,S&P 500 E-micro,@MES,MES,GLOBEX,0.000033,0.47,1.25,0.25,5,2841.0,14205,1320.00,1200.00
2,Nasdaq 100 E-micro,@MNQ,MNQ,GLOBEX,0.000026,0.47,0.50,0.25,2,9038.0,18076,1652.81,1502.55
3,Dow Futures micro,@MYM,MYM,CBT,0.000020,0.47,0.50,0.50,1,23597.0,23597,3608.00,3137.39
4,Russell 2000 E-micro,@M2K,M2K,GLOBEX,0.000075,0.47,0.50,0.10,5,1257.0,6285,980.45,784.36


In [4]:
iqfeed_1min_data_folder = r"D:\data\iqfeed\future\1min_raw"
vnpy_1min_data_folder = r"D:\data\vnpy\future\1_min"

In [5]:
iqfeed_data_path_list = glob.glob(os.path.join(iqfeed_1min_data_folder, "*.csv"))

In [6]:
__YEARS__ = ["06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"]
__MONTH_CODES__ = ["F", "G", "H", "J", "K", "M", "N", "Q", "U", "V", "X", "Z"]

for ix, row in symbol_frame.iterrows():
    iqfeed_root = row.loc["iqfeed_roots"]
    ib_root = row.loc["ib_root"]
    exchange = row.loc["symbol_exchange"]
    for year in __YEARS__:
        for month_code in __MONTH_CODES__:
            symbol = "".join([iqfeed_root, month_code, year])
            symbol_csv_string = "".join([symbol, "_1.csv"])
            symbol_csv_path = os.path.join(iqfeed_1min_data_folder, symbol_csv_string)
            ib_symbol = "".join([ib_root, month_code, year])
            if symbol_csv_path in iqfeed_data_path_list:
                imported_data = pd.read_csv(symbol_csv_path)
                imported_data.columns = ["datetime", "open", "high", "low", "close", "volume"]
                imported_data.loc[:, "exchange"] = Exchange(exchange)
                imported_data.loc[:, "open_interest"] = 0.0
                imported_data.loc[:, "interval"] = Interval.MINUTE
                imported_data.loc[:, "symbol"] = ib_symbol
                datetime_format = '%Y%m%d %H:%M:%S'
                imported_data['datetime'] = pd.to_datetime(imported_data['datetime'], format=datetime_format)
                float_columns = ['open', 'high', 'low', 'close', 'volume', 'open_interest']
                for col in float_columns:
                    imported_data.loc[:, col] = imported_data[col].astype('float')
                imported_data.to_csv(os.path.join(vnpy_1min_data_folder, ib_symbol + ".csv"), index=False)